- has function , add_ingestion date, re_order_df_cols, load_incremental_data

In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(in_df):
    out_df = in_df.withColumn("ingestion_date", current_timestamp())
    return out_df

In [0]:
#method 1
# Filter the list into two parts: one without 'id' and the other with only 'id'.
# my_list = [1, 2, 3, 4, 5, 6, 'id', 8, 9]
# Concatenate these two parts, ensuring 'id' is at the end.
# output = [1, 2, 3, 4, 5, 6, 8, 9, 'id']

def re_order_df_cols(input_df, partation_col):
    ordered_cols = []
    for col_name in input_df.schema.names:
        if col_name != partation_col:
            ordered_cols.append(col_name)
    ordered_cols.append(partation_col)
    output_df = input_df.select(ordered_cols)
    return output_df


In [0]:
# function to load incremental data (other methods are available in incremental load> results_ingestion script)
# parameters - input_df, partation_col, db_name, table_name
def load_incremental_data(input_df, partation_col, db_name, table_name):
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    ordered_df = re_order_df_cols(input_df, partation_col)
    if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
        ordered_df.write.mode("overwrite").insertInto(f"{db_name}.{table_name}") # we do not have a option to specify partition in insertInto, by default spark will consider last columnof df as partation column so need to adjust schema so that race is comes last using select in above cell
    else:
        ordered_df.write.mode("overwrite").partitionBy(f"{partation_col}").format("parquet").saveAsTable(f"{db_name}.{table_name}")

In [0]:
def merge_incremental_data(input_df, folder_path,merge_condition, partation_col, db_name, table_name):
    spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
    from delta.tables import DeltaTable
    if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
        deltaTable = DeltaTable.forPath(spark, f"{folder_path}/{table_name}")
        deltaTable.alias("target_tab")\
            .merge(input_df.alias("source_tab"),\
            merge_condition)\
            .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
            .execute()
    else:
        input_df.write.mode("overwrite").partitionBy(f"{partation_col}").format("delta").saveAsTable(f"{db_name}.{table_name}")